### Bayes Networks
The assumption is we have a directed graph such that if a node $A$ has no parents we are given the probability distribution $P(A)$, and if instead $A$ has a set of parent nodes $S$ then we are given the conditional distribution $P(A|S)$.

For now we assume these are boolean random variables. Below I aim to implement an algorithm to compute a particular probability for a particular graph, and and algorithm to approximate this by sampling. Hopefully this will give me some ideas about how to best set this up in general.

The graph will be a simple 5 node graph, nodes A and B lead into node C, and node C leads into both node D and node E.

For each node we need to store a probability distribution and the children nodes. A simple way to do this is with dictionaries, though in general there should most likely be a node class and a probability distriution class.

First are the probabilities.

In [1]:
# Probability A is true
def probA():
    return 0.5
# Probability B is true
def probB():
    return 0.25
# Probability C is true given A and B
def probC(a,b):
    if a and b:return 0.9
    elif a and not b: return 0.6
    elif not a and b: return 0.75
    elif not a and not b: return 0.1
# Probability D is true given C
def probD(c):
    if c: return 0.6
    elif not c: return 0.4
# Probability E is true given C
def probE(c):
    if c: return 0.75
    elif not c: return 0.25

##### Remarks
In general if a node has $n$ parents that can take on Boolean values you would need to define $2^n$ probabilities. There may be some way to store all these at once and return this instead.

Next are the nodes.

In [6]:
# simple node class
class Node:
    # initialise a node with its probability distribution and children nodes
    def __init__(self,name,probability_distribution, children):
        self.name = name
        self.prob = probability_distribution
        self.children = children

In [7]:
# Notice these need to be defined from the 'youngest' nodes
# i.e. the leaf nodes.
# That may be problematic in general.
E = Node('E', probE, None)
D = Node('D', probD, None)
C = Node('C', probC, [D,E])
B = Node('B', probB, [C])
A = Node('A', probA, [C])

#### Task
Compute the following conditional

$$ P(A=T, B=T | D=T, E=T). $$

Let $P(A)$ denote $P(A=T)$. Then we know

$$ P(A, B |D,E) = \frac{P(A,B,D,E)}{P(D,E)}$$

We can factor the numerator as

$$P(A,B,C,D,E) = \sum_{c} P(D,E|C=c)P(C=c|A,B)P(A)P(B)$$

Similarly the denominator can be factored as 

$$ P(D,E) = \sum_{a,b,c} P(D,E |C=c)P(C=c |A=a,B=b)P(A=a)P(B=b). $$

The key difference is $A$ and $B$ can be false in the denominator.

Below is a function which computes these sums.

In [16]:
def conditional_on_AB(a,b):
    # priors
    if a: pa = A.prob()
    else: pa = 1-A.prob()
    if b: pb = B.prob()
    else: pb = 1-B.prob()
    
    # conditionals
    sum = 0
    for i in [True, False]:
        if i: c = C.prob(a,b)
        else: c = 1-C.prob(a,b)
        sum+=D.prob(i)*E.prob(i)*c*pa*pb
    return sum

# the numerator is conditional_on_AB(True,True)
numerator = conditional_on_AB(True,True)
# the denominator is the sum over all combinations
denominator = 0
for i in [True,False]:
    for j in [True,False]:
        denominator+=conditional_on_AB(i,j)

print("The probability that A and B are true given D and E are is {}".format(numerator/denominator))

The probability that A and B are true given D and E are is 0.19644970414201185


##### Remarks
There's almost certainly a better way to do this!

Next this probability can be estimated by sampling.

In [29]:
import random
import time
def generate_sample():
    x = random.random()
    if(x<A.prob()):a = True
    else: a=False
    x = random.random()
    if(x<B.prob()):b= True
    else: b=False
    x = random.random()
    if(x<C.prob(a,b)): c = True
    else : c = False
    x = random.random()
    if x<D.prob(c):d = True
    else: d = False
    x = random.random()
    if x<E.prob(c):e = True
    else: e=False
    return a,b,c,d,e

start = time.time()
# compute frequency of samples where a,b are true when d,e are true
numerator = 0
denominator = 0
num_iter = 1000000
for i in range(num_iter):
    a,b,c,d,e = generate_sample()
    if d and e:
        denominator+=1
        if a and b:
            numerator+=1
stop = time.time()
print("The sample approximation to the probability that A and B are true, given D and E are, is {}".format(numerator/denominator))
print("This took {}s".format(stop-start))

The sample approximation to the probability that A and B are true, given D and E are, is 0.19576353525157553
This took 2.3401339054107666s


Many samples are generated only to be dropped in the first conditional. Likelihood weighting only generates useful samples for the inference task at hand, by weighting the samples appropriately. In this case the weight is $P(D|C=c)P(E|C=c)$.

In [30]:
def generate_likelihood_sample():
    x = random.random()
    if(x<A.prob()):a = True
    else: a=False
    x = random.random()
    if(x<B.prob()):b= True
    else: b=False
    x = random.random()
    if(x<C.prob(a,b)): c = True
    else : c = False
    return a,b,c, D.prob(c)*E.prob(c)

# compute weighted frequency of samples where a,b are true
start = time.time()
numerator = 0
denominator = 0
num_iter = 1000000
for i in range(num_iter):
    a,b,c,weight = generate_likelihood_sample()
    denominator+=weight
    if a and b:
        numerator+=weight
end = time.time()
print("The sample approximation to the probability that A and B are true, given D and E are, is {}".format(numerator/denominator))
print("This took {}s".format(end-start))

The sample approximation to the probability that A and B are true, given D and E are, is 0.19725220971704557
This took 1.9941141605377197s


#### Remarks
Even in this simple case the approximation methods are only accurate for the first two decimal places, even after many iterations. The likelihood weighted sampling is slightly quicker in this case, and would be even quicker than plain sampling in the case that the original conditioned on event (in this case $D,E$ being true) was rare.